# Senti preprocess 2 使用的

In [8]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Yelp'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size', 'Compatibility', 'Longevity']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']
elif meta_dataset == 'Home_and_Kitchen':
    # Price, Brand, Quality, Material, Size, Appearance, Washability, Functionality, Style, Durability,
    attr_list = ['Price', 'Brand', 'Quality', 'Material', 'Size', 'Appearance', 'Washability', 'Functionality', 'Style']
elif meta_dataset == 'Sports_and_Outdoors':
    # Price, Quality, Functionality,Size,Protection,Material,Comfort
    attr_list = ['Price', 'Quality', 'Functionality', 'Size', 'Protection', 'Material', 'Comfort']
elif meta_dataset == 'Yelp':
    # Price, Brand, Quality, Material, Size, Appearance, Washability, Functionality, Style, Durability,
    attr_list = ['Price', 'Food', 'Location', 'Ambience', 'Service', 'Cleanliness', 'Parking']

data_all = pd.read_csv(data_path)
if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
    data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6],attr_list[7],attr_list[8]]]
else:
    data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]

attr_num = len(attr_list)

# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}
attr1_all = {}
attr2_all = {}
attr3_all = {}
attr4_all = {}
attr5_all = {}
attr6_all = {}
attr7_all = {}
attr8_all = {}
attr9_all = {}

item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    attr1_opin = row[attr_list[0]]
    attr2_opin = row[attr_list[1]]
    attr3_opin = row[attr_list[2]]
    attr4_opin = row[attr_list[3]]
    attr5_opin = row[attr_list[4]]
    attr6_opin = row[attr_list[5]]
    attr7_opin = row[attr_list[6]]
    if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
        attr8_opin = row[attr_list[7]]
        attr9_opin = row[attr_list[8]]
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id].append(attr7_opin)
        if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
            attr8_all[sess_id].append(attr8_opin)
            attr9_all[sess_id].append(attr9_opin)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id] = []
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id] = []
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id] = []
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id] = []
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id] = []
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id] = []
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id] = []
        attr7_all[sess_id].append(attr7_opin)
        if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
            attr8_all[sess_id] = []
            attr9_all[sess_id] = []
            attr8_all[sess_id].append(attr8_opin)
            attr9_all[sess_id].append(attr9_opin)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        attr1_all[sess_id] = attr1_all[sess_id][-20:]
        attr2_all[sess_id] = attr2_all[sess_id][-20:]
        attr3_all[sess_id] = attr3_all[sess_id][-20:]
        attr4_all[sess_id] = attr4_all[sess_id][-20:]
        attr5_all[sess_id] = attr5_all[sess_id][-20:]
        attr6_all[sess_id] = attr6_all[sess_id][-20:]
        attr7_all[sess_id] = attr7_all[sess_id][-20:]
        if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
            attr8_all[sess_id] = attr8_all[sess_id][-20:]
            attr9_all[sess_id] = attr9_all[sess_id][-20:]

# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u_id ...], [x_t-1...], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}

# dict {item_id:opin_id}
attr1_item_opin = {}
attr2_item_opin = {}
attr3_item_opin = {}
attr4_item_opin = {}
attr5_item_opin = {}
attr6_item_opin = {}
attr7_item_opin = {}
attr8_item_opin = {}
attr9_item_opin = {}


for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    a1_opin = attr1_all[sess_temp][:-1]
    a2_opin = attr2_all[sess_temp][:-1]
    a3_opin = attr3_all[sess_temp][:-1]
    a4_opin = attr4_all[sess_temp][:-1]
    a5_opin = attr5_all[sess_temp][:-1]
    a6_opin = attr6_all[sess_temp][:-1]
    a7_opin = attr7_all[sess_temp][:-1]
    if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
        a8_opin = attr8_all[sess_temp][:-1]
        a9_opin = attr9_all[sess_temp][:-1]
    else:
        a8_opin = attr6_all[sess_temp][:-1]
        a9_opin = attr7_all[sess_temp][:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x, a1, a2, a3, a4, a5, a6, a7, a8, a9 in zip(train_seqs_t, a1_opin, a2_opin,a3_opin,a4_opin,a5_opin,a6_opin,a7_opin,a8_opin,a9_opin):
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
                attr1_item_opin[item_dict[x]].append(a1)
                attr2_item_opin[item_dict[x]].append(a2)
                attr3_item_opin[item_dict[x]].append(a3)
                attr4_item_opin[item_dict[x]].append(a4)
                attr5_item_opin[item_dict[x]].append(a5)
                attr6_item_opin[item_dict[x]].append(a6)
                attr7_item_opin[item_dict[x]].append(a7)
                if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
                    attr8_item_opin[item_dict[x]].append(a8)
                    attr9_item_opin[item_dict[x]].append(a9)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            attr1_item_opin[item_num] = []
            attr2_item_opin[item_num] = []
            attr3_item_opin[item_num] = []
            attr4_item_opin[item_num] = []
            attr5_item_opin[item_num] = []
            attr6_item_opin[item_num] = []
            attr7_item_opin[item_num] = []
            
            attr1_item_opin[item_num].append(a1)
            attr2_item_opin[item_num].append(a2)
            attr3_item_opin[item_num].append(a3)
            attr4_item_opin[item_num].append(a4)
            attr5_item_opin[item_num].append(a5)
            attr6_item_opin[item_num].append(a6)
            attr7_item_opin[item_num].append(a7)
            if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
                attr8_item_opin[item_num] = []
                attr9_item_opin[item_num] = []
                attr8_item_opin[item_num].append(a8)
                attr9_item_opin[item_num].append(a9)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
#     预测时考虑last item
    tra[1].append(new_seq[:-1])
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

fre_item_mat = np.zeros((item_count, item_count), dtype=int)
# 全部的共现关系Original 
for sess in train_sess.keys():
    item_seq = np.unique(train_sess[sess])
    for it1 in item_seq:
        for it2 in item_seq:
            if it1 != it2:
#                 print(sess)
                fre_item_mat[int(it1)-1, int(it2)-1] +=1 
                
fre_user_mat = np.zeros((user_count, user_count), dtype=int)
# 全部的共现关系Original 
for item_t in train_items.keys():
    user_seq = np.unique(train_items[item_t])
    for it1 in user_seq:
        for it2 in user_seq:
            if it1 != it2:
                fre_user_mat[int(it1)-1, int(it2)-1] +=1 

# building user_fre & item_fre csr_matrix
def spar_graph(fre_mat):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for i_list in fre_mat:
        i_temp_list = np.nonzero(i_list)[0].tolist()
#         if len(i_temp_list) == 0:
#             non_co += 1
        indices += i_temp_list
        all_data += [1]*len(i_temp_list)
        indptr.append(indptr[-1]+len(i_temp_list))
#     print('item without co-occurren: ',str(non_co))
    # indptr:session长度累加和; indices:item_id 减1, 由每个session内item组成; data:item在session内的权重，全部为1.
    coo_mat = (all_data, indices, indptr)
    return coo_mat

item_co_mat = spar_graph(fre_item_mat)
user_co_mat = spar_graph(fre_user_mat)


# test_set [[u_id ...], [x_t-1...], [label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        tes_seq_num += 1
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes[0].append(sess_temp)
    #     预测时考虑last item
        tes[1].append(new_seq[:-1])
        tes[2].append(item_dict[all_seqs[-1]])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num

#  obtain user-item interaction sparsity matrix in each attribute
# def spar_matrix(attr, tra_seq):
#     indices = [] # 非0元素的列坐标，item_id -1
#     all_data = [] # 非0元素值，opin_id
#     indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
#     for sess_temp in attr.keys():
#         op_list = attr[sess_temp][:-1]
#         it_list = tra_seq[sess_temp]
#         np_op = np.array(op_list)
#         np_it = np.array(it_list)
#         op_f = np_op[np.nonzero(np_op)[0]]
#         it_f = np_it[np.nonzero(np_op)[0]]

#         item_l = np.unique(it_f)
#         op_l = op_f[np.unique(it_f, return_index=True)[1]]
#         indices += (item_l-1).tolist()
#         all_data += op_l.tolist()
#         indptr.append(indptr[-1] + len(item_l.tolist()))
#     results = (all_data, indices, indptr)
#     return results

# attr1_mat = spar_matrix(attr1_all, train_sess)
# attr2_mat = spar_matrix(attr2_all, train_sess)
# attr3_mat = spar_matrix(attr3_all, train_sess)
# attr4_mat = spar_matrix(attr4_all, train_sess)
# attr5_mat = spar_matrix(attr5_all, train_sess)
# attr6_mat = spar_matrix(attr6_all, train_sess)
# attr7_mat = spar_matrix(attr7_all, train_sess)

# def spar_matrix(tra_seq):
#     indices = [] # 非0元素的列坐标，item_id -1
#     all_data = [] # 非0元素值，opin_id
#     indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
#     for sess_temp in tra_seq.keys():
#         op_list = attr[sess_temp][:-1]
#         it_list = tra_seq[sess_temp]
#         np_op = np.array(op_list)
#         np_it = np.array(it_list)
#         op_f = np_op[np.nonzero(np_op)[0]]
#         it_f = np_it[np.nonzero(np_op)[0]]

#         item_l = np.unique(it_f)
#         op_l = op_f[np.unique(it_f, return_index=True)[1]]
#         indices += (item_l-1).tolist()
#         all_data += op_l.tolist()
#         indptr.append(indptr[-1] + len(item_l.tolist()))
#     results = (all_data, indices, indptr)
#     return results

def spar_matrix(tra_seq):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for sess_temp in tra_seq.keys():
        it_list = tra_seq[sess_temp]
        it_index = np.unique(it_list) - 1
        indices += (it_index).tolist()
        all_data += [1]*len(it_index)
        indptr.append(indptr[-1] + len(it_index.tolist()))
    results = (all_data, indices, indptr)
    return results

ui_co_mat = spar_matrix(train_sess)




# 返回 [[u1,u2],[attr1_opin, attr1_opin]]
def formulate_seq(dict_ui, dict_op):
    results = [[],[]]
    temp_res = [[],[]]
    max_len =0
    for x in dict_ui.keys():
        seqs_id = dict_ui[x]
        opin_id = dict_op[x]
        new_us_seq = []
        new_op_seq = []
        for id_t, op_t in zip(seqs_id, opin_id):
            if op_t != 0 and id_t not in new_us_seq:
                new_us_seq.append(id_t)
                new_op_seq.append(op_t)
        if len(new_us_seq)==0:
            new_us_seq.append(0)
            new_op_seq.append(0)
        if len(new_us_seq)>max_len:
            max_len = len(new_us_seq)
        temp_res[0].append(new_us_seq)
        temp_res[1].append(new_op_seq)
    for ui_seq, op_seq in zip(temp_res[0],temp_res[1]):
        pad_len = max_len - len(ui_seq)
        pad_ui_seq = ui_seq + [0]*pad_len
        pad_op_seq = op_seq + [0]*pad_len
        results[0].append(pad_ui_seq)
        results[1].append(pad_op_seq)
#     print('max_len:',str(max_len))
    return results

# u_seq = {1:[1,2,3,1],2:[4,5], 3:[7,8,9]}
# u_op = {1:[45,56,0,45],2:[66,0], 3:[0,33,0]}
# # [[[1,2],[4],[8]],[[45,56],[66],[33]]]
# formulate_seq(u_seq,u_op)

ui_1 = formulate_seq(train_sess,attr1_all)
ui_2 = formulate_seq(train_sess,attr2_all)
ui_3 = formulate_seq(train_sess,attr3_all)
ui_4 = formulate_seq(train_sess,attr4_all)
ui_5 = formulate_seq(train_sess,attr5_all)
ui_6 = formulate_seq(train_sess,attr6_all)
ui_7 = formulate_seq(train_sess,attr7_all)
iu_1 = formulate_seq(train_items,attr1_item_opin)
iu_2 = formulate_seq(train_items,attr2_item_opin)
iu_3 = formulate_seq(train_items,attr3_item_opin)
iu_4 = formulate_seq(train_items,attr4_item_opin)
iu_5 = formulate_seq(train_items,attr5_item_opin)
iu_6 = formulate_seq(train_items,attr6_item_opin)
iu_7 = formulate_seq(train_items,attr7_item_opin)

if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
    ui_8 = formulate_seq(train_sess,attr8_all)
    ui_9 = formulate_seq(train_sess,attr9_all)
    iu_8 = formulate_seq(train_items,attr8_item_opin)
    iu_9 = formulate_seq(train_items,attr9_item_opin)

def ui_seq(dict_ui):
    results = []
    temp_res = []
    max_len =0
    for x in dict_ui.keys():
        seqs_id = dict_ui[x]
        if len(seqs_id)>max_len:
            max_len = len(seqs_id)
        temp_res.append(seqs_id)
    for temp_l in temp_res:
        pad_len = max_len - len(temp_l)
        pad_ui_seq = temp_l + [0]*pad_len
        results.append(pad_ui_seq)
    return results

ui_list = ui_seq(train_sess)
iu_list = ui_seq(train_items)

print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './datasets/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"

tra_save = (tra[0], tra[1], tra[2],item_co_mat,user_co_mat, ui_list, iu_list, ui_1, ui_2, ui_3, ui_4, ui_5, ui_6, ui_7, iu_1, iu_2, iu_3, iu_4, iu_5, iu_6, iu_7)
tes_save = (tes[0], tes[1], tes[2])

if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
    tra_save = (tra[0], tra[1], tra[2],item_co_mat,user_co_mat, ui_list, iu_list, ui_1, ui_2, ui_3, ui_4, ui_5, ui_6, ui_7,ui_8, ui_9, iu_1, iu_2, iu_3, iu_4, iu_5, iu_6, iu_7, iu_8, iu_9)

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

item_num:  12391
train_seq_num & user_num:  12373
test_seq_num:  12311
#interactions:  110313
sequence average length:  8.915622726905358
dataset:  Yelp
2023-12-29 08:51:17
done


[4, 5, 6]

## NARM

In [9]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Yelp'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']
elif meta_dataset == 'Sports_and_Outdoors':
    # Price, Quality, Functionality,Size,Protection,Material,Comfort
    attr_list = ['Price', 'Quality', 'Functionality', 'Size', 'Protection', 'Material', 'Comfort']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID']]
# data_all = data_all[['sessionID', 'itemID']]


# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}


item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all: 
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[x1, x2, x3], [label...]
tra = [[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(new_seq[:-1])
#     预测时考虑last item
    tra[1].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(new_seq[:-1])
    #     预测时考虑last item
        tes[1].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './NARM/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
tra_save = (tra[0], tra[1])
tes_save = (tes[0], tes[1])

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("NARM dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")


item_num:  12391
train_seq_num & user_num:  12373
test_seq_num:  12311
#interactions:  110313
sequence average length:  8.915622726905358
NARM dataset:  Yelp
2023-12-29 08:51:52
done


## BERT4Rec

In [ ]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Cell_Phones_and_Accessories'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data_all = data_all[['sessionID', 'itemID']]


# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}


item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[user1,user2],[x1, x2, x3], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[user1,user2],[x_id ...],[label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './BERT4Rec/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
tra_save = (tra[0], tra[1], tra[2])
tes_save = (tes[0], tes[1], tes[2])

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("BERT4Rec dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")


## UniSRec

In [10]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Yelp'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'text']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}
#  dict {old_itemID: title}
olditem_title = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if item_id not in olditem_title:
        olditem_title[item_id] = row['text']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
item_num = 0
# dict{lod_itemID:new_itemID}
item_dict = {}
# dict {new_item_id:text}
newitem_title = {}
# train_set [[u1, u2],[x1, x2, x3], [label...]
tra = [[],[],[]]
val = [[], [], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            newitem_title[item_num] = olditem_title[x]
            new_seq.append(item_num)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    val[0].append(sess_temp)
    val[1].append(new_seq[:-2])
#     预测时考虑last item
    val[2].append(new_seq[-2])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

train_data_path = './UniSRec/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_train = train_data_path + f'{datasets_name}Senti.train.inter'
path_data_valid = train_data_path + f'{datasets_name}Senti.valid.inter'
path_data_test = train_data_path + f'{datasets_name}Senti.test.inter'
# tra[0], tra[1], tra[2]
# tes[0], tes[1], tes[2]
# split_valid = int(len(tra[2])/9*7)
# train_sess = tr_seqs[:split_valid]
# train_labs = tr_labs[:split_valid]
# valid_sess = tr_seqs[split_valid:]
# valid_labs = tr_labs[split_valid:]

with open(path_data_train, 'w') as file:
    file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for index, sess, lab in zip(tra[0], tra[1], tra[2]):
        i_str = str(index)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')
                   
with open(path_data_valid, 'w') as file:
    file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for index, sess, lab in zip(val[0], val[1], val[2]):
        i_str = str(index)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')
                   
with open(path_data_test, 'w') as file:
    file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for index, sess, lab in zip(tes[0], tes[1], tes[2]):
        i_str = str(index)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')

uni_text_path = train_data_path + 'textlist.npy'
np.save(uni_text_path, list(newitem_title.values()))                   

print("dataset: ", datasets_name)
print("model: UniSRec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

item_num:  12390
train_seq_num & user_num:  12373
test_seq_num:  12311
#interactions:  110313
sequence average length:  8.915622726905358
dataset:  Yelp
model: UniSRec
2023-12-29 08:52:16
done


## MCLRec 

In [11]:
import os
import pickle
import time
import pandas as pd
import numpy as np
import json

datasets_name = 'Yelp'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = int(row['sessionID'])
    item_id = row['itemID']
    
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
item_num =1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u1, u2],[x1, x2, x3], [time...]
tra = [[],[]]
# val = [[], [], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in all_seqs:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0] += [sess_temp]*len(new_seq)
    tra[1] += new_seq

    


train_data_path = './MCLRec/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_test = train_data_path + datasets_name + '.txt'


# tra_dict = {'user_id':tra[0], 'item_id':tra[1], 'timestamp':tra[2]}
# tra_df = pd.DataFrame(tra_dict)
       
# tes_dict = {'user_id':tes[0], 'item_id':tes[1], 'timestamp':tes[2]}
# tes_df = pd.DataFrame(tes_dict)

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))

path_inter = train_data_path + f'{datasets_name}.inter'


with open(path_inter, 'w') as file:
    file.write('user_id:token\titem_id:token\n')
    for u_id, i_id in zip(tra[0], tra[1]):
        u_str = str(u_id)
        i_str = str(i_id)
        file.write(f'{u_str}\t{i_str}\n')  
        
with open(path_data_test, 'w') as file:
    for sess_temp in train_sess.keys():
        u_str = str(sess_temp)
        sess = train_sess[sess_temp]
        s_str = [str(x) for x in sess]
        file.write(f'{i_str} {" ".join(s_str)}\n')

print("dataset: ", datasets_name)
print("model: MCLRec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

dataset:  Yelp
model: MCLRec
2023-12-29 08:52:54
done


## ACTSR & FEARec

In [12]:
import os
import pickle
import time
import pandas as pd
import numpy as np
import json

datasets_name = 'Yelp'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = int(row['sessionID'])
    item_id = row['itemID']
    
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
item_num =1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u1, u2],[x1, x2, x3], [time...]
tra = [[],[]]
# val = [[], [], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in all_seqs:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0] += [sess_temp]*len(new_seq)
    tra[1] += new_seq

    


train_data_path = './ACTSR/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_test = train_data_path + datasets_name + '.txt'


# tra_dict = {'user_id':tra[0], 'item_id':tra[1], 'timestamp':tra[2]}
# tra_df = pd.DataFrame(tra_dict)
       
# tes_dict = {'user_id':tes[0], 'item_id':tes[1], 'timestamp':tes[2]}
# tes_df = pd.DataFrame(tes_dict)

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))

path_inter = train_data_path + f'{datasets_name}.inter'


with open(path_inter, 'w') as file:
    file.write('user_id:token\titem_id:token\ttimestamp:token\n')
    for u_id, i_id, t_time in zip(tra[0], tra[1], range(len(tra[0]))):
        u_str = str(u_id)
        i_str = str(i_id)
        t_str = str(t_time)
        file.write(f'{u_str}\t{i_str}\t{t_str}\n')  
        
with open(path_data_test, 'w') as file:
    for sess_temp in train_sess.keys():
        u_str = str(sess_temp)
        sess = train_sess[sess_temp]
        s_str = [str(x) for x in sess]
        file.write(f'{i_str} {" ".join(s_str)}\n')

print("dataset: ", datasets_name)
print("model: ACTSR")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

dataset:  Yelp
model: ACTSR
2023-12-29 08:53:08
done


## RNS

In [13]:
import os
import pickle
import time
import pandas as pd
import numpy as np
import json

datasets_name = 'Yelp'
# Cell_Phones_and_Accessories/
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID','review_time', 'reviewText']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}

# dict {sessionID:reviewtext1}
u_review = {}
# dict {sessionID:[time,time]}
# u_i_time = {}
#  dict {old_itemID: review}
olditem_review = {}
for _, row in data.iterrows():
    sess_id = int(row['sessionID'])-1
    item_id = row['itemID']
    review_t = row['reviewText']
    time_t = row['review_time']
    if item_id not in olditem_review:
        olditem_review[item_id] = review_t
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        u_review[sess_id] = review_t

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
item_num = 0
# dict{lod_itemID:new_itemID}
item_dict = {}
# dict {new_item_id:text}
newitem_review = {}
# train_set [[u1, u2],[x1, x2, x3], [label...]
tra = [[],[],[],[]]
# val = [[], [], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            newitem_review[item_num] = olditem_review[x]
            new_seq.append(item_num)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append([sess_temp]*len(new_seq))
    tra[1].append(new_seq)
    tra[2].append([1]*len(new_seq))
    tra[3].append([2]*len(new_seq))
#     预测时考虑last item
    
item_count =  item_num
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        tes[0].append(sess_temp)
        tes[1].append(item_dict[all_seqs[-1]])
        tes[2].append(1)
        tes[3].append(2)
    else:
        tes[0].append(sess_temp)
        tes[1].append(0)
        tes[2].append(1)
        tes[3].append(2)
    



word_dict = {}
word_num = 0
u_num = {}
max_review_word = 30
for u_t in u_review.keys():
    rev = u_review[u_t].split(' ')
    if len(rev) == 0:
        u_num[u_t] = [[0]]
    else:
        u_num[u_t] = []
        wor_l = []
        for word_t in rev:
            if word_t in word_dict:
                wor_l.append(word_dict[word_t])
            else:
                word_dict[word_t] = word_num
                wor_l.append(word_num)
                word_num +=1
        if len(wor_l) < max_review_word:
            wor_l = wor_l + [0]*(max_review_word-len(wor_l))
        else:
            wor_l = wor_l[:max_review_word]
        u_num[u_t].append(wor_l)
        
word_dict[0]='a12sasa2'
                    

i_num = {}
for i_t in newitem_review.keys():
    rev = newitem_review[i_t].split(' ')
    if len(rev) == 0:
        i_num[i_t] = [[0]]
    else:
        i_num[i_t] = []
        wor_l = []
        for word_t in rev:
            if word_t in word_dict:
                wor_l.append(word_dict[word_t])
            else:
                word_dict[word_t] = word_num
                wor_l.append(word_num)
                word_num +=1
        if len(wor_l) < max_review_word:
            wor_l = wor_l + [0]*(max_review_word-len(wor_l))
        else:
            wor_l = wor_l[:max_review_word]
        i_num[i_t].append(wor_l)

print('word number:',str(word_num-1))

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

train_data_path = './RNS/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_train = train_data_path + 'train.csv'
path_data_test = train_data_path + 'test.csv'
path_i_text = train_data_path + 'i_text'
path_u_text = train_data_path + '/u_text'
path_voc = train_data_path + 'vocabulary'


with open(path_data_train, 'w') as file:
#     file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for u,i, r, t in zip(tra[0], tra[1], tra[2], tra[3]):
        for u_t, i_t, r_t, t_t in zip(u,i,r,t):
            str_1 = str(u_t)
            str_2 = str(i_t)
            str_3 = str(r_t)
            str_4 = str(t_t)
            file.write(f'{str_1},{str_2},{str_3},{str_4}\n')
                   
                   
with open(path_data_test, 'w') as file:
    for u_t, i_t, r_t, t_t in zip(tes[0], tes[1], tes[2], tes[3]):
        str_1 = str(u_t)
        str_2 = str(i_t)
        str_3 = str(r_t)
        str_4 = str(t_t)
        file.write(f'{str_1},{str_2},{str_3},{str_4}\n')
            
with open(path_i_text, "wb") as file_object:
    pickle.dump(i_num, file_object)
with open(path_u_text, "wb") as file_object:
    pickle.dump(u_num, file_object)            

with open(path_voc, "wb") as file_object:
    pickle.dump(word_dict, file_object) 
print("dataset: ", datasets_name)
print("model: RNS")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

word number: 82627
item_num:  12391
train_seq_num & user_num:  12373
test_seq_num:  0
#interactions:  98002
sequence average length:  7.920633637759638
dataset:  Yelp
model: RNS
2023-12-29 08:53:32
done


## SKNN

In [14]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Yelp'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'

# datasets_name = '2019-Oct'
# price_num = 'logistic'

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'

# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'


print("******SKNN******")

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'review_time']]


# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})[['itemID', 'item_num']]
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID', 'time']]
 

# dict (sessionID:[itemID,itemID])
sess_all = {}
sess_date = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    timestamp = row['review_time']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        sess_date[sess_id].append(timestamp)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = []        
        sess_date[sess_id].append(timestamp)
        
# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    time_seq = sess_date[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        sess_date[sess_temp] = time_seq[-20:]

tra = [[],[],[], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}
item_dict = {}
item_count = 0
user_count = 0
inter_count = 0
item_num = 1

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    t_time_seq = sess_date[sess_temp][:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    tra[3].append(t_time_seq)
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[], []]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    all_time = sess_date[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
        tes[3].append(all_time)
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

tra = (tra[0], tra[1], tra[2], tra[3])
tes = (tes[0], tes[1], tes[2], tes[3])

train_data_path = './SKNN/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("SKNN done")

******SKNN******
item_num:  12391
train_seq_num & user_num:  12373
test_seq_num:  12311
#interactions:  110313
sequence average length:  8.915622726905358
dataset:  Yelp
2023-12-29 08:53:55
SKNN done


1

## SSLRec Fine

In [15]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Yelp'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}
#  dict {old_itemID: title}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u1, u2],[x1, x2, x3], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

train_data_path = 'SSLRecFine/' + datasets_name + 'Fine/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)


path_data_train = train_data_path + 'train.tsv'
# path_data_valid = train_data_path + f'{datasets_name_c}.valid.inter'
path_data_test = train_data_path + 'test.tsv'
# train_sessid_list = range(len(tr_labs))
with open(path_data_train, 'w') as file:
    file.write('session_id\t item_id_seq\t item_id \n')
    for index, sess, lab in zip(tra[0],tra[1], tra[2]):
        i_str = str(index-1)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')


with open(path_data_test, 'w') as file:
    file.write('session_id\t item_id_seq\t item_id \n')
    for index, sess, lab in zip(range(len(tes[0])),tes[1], tes[2]):
        i_str = str(index-1)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')


# train_sessid_list = range(len(tr_labs))
# train_seq_list = tr_seqs
# train_lab_list = tr_labs
# train_pd = pd.DataFrame({'session_id':train_sessid_list, 'item_id_seq':train_seq_list, 'item_id':train_lab_list})

# test_sessid_list = range(len(te_labs))
# test_seq_list = te_seqs
# test_lab_list = te_labs
# test_pd = pd.DataFrame({'session_id':test_sessid_list, 'item_id_seq':test_seq_list, 'item_id':test_lab_list})

# train_pd.to_csv(path_data_train, sep = '\t', index=False)
# test_pd.to_csv(path_data_test, sep = '\t', index=False)

print("dataset: ", datasets_name)
print("model: SSLRec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

item_num:  12391
train_seq_num & user_num:  12373
test_seq_num:  12311
#interactions:  110313
sequence average length:  8.915622726905358
dataset:  Yelp
model: SSLRec
2023-12-29 09:23:49
done


## AutoSecRec

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np
import json

datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID','review_time']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}
# dict{sessionID:[time]}
time_all = {}
for _, row in data.iterrows():
    sess_id = int(row['sessionID'])-1
    item_id = row['itemID']
    time_t = row['review_time']
    
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        time_all[sess_id].append(time_t)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        time_all[sess_id] = []
        time_all[sess_id].append(time_t)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        time_all[sess_temp] = time_all[sess_temp][-20:]
        
item_num = 0
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u1, u2],[x1, x2, x3], [time...]
tra = [[],[],[]]
# val = [[], [], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    train_time_t = time_all[sess_temp][:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0] += [sess_temp]*len(new_seq)
    tra[1] += new_seq
    tra[2] += train_time_t
#     预测时考虑last item
    
item_count =  item_num
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        tra_seqs = train_sess[sess_temp]
        tes[0] += [sess_temp]*(len(tra_seqs)+1)
        tes[1] += tra_seqs + [(item_dict[all_seqs[-1]])]
        tes[2] += range(len(tra_seqs)+1)
        tes_seq_num +=1
        
#         tes[0].append(sess_temp)
#         tes[1].append(item_dict[all_seqs[-1]])
#         tes[2].append(time_all[sess_temp][-1])
#         tes_seq_num +=1

    



print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

train_data_path = './AutoSecRec/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_train = train_data_path + 'train.txt'
path_data_test = train_data_path + 'test.txt'


# tra_dict = {'user_id':tra[0], 'item_id':tra[1], 'timestamp':tra[2]}
# tra_df = pd.DataFrame(tra_dict)
       
# tes_dict = {'user_id':tes[0], 'item_id':tes[1], 'timestamp':tes[2]}
# tes_df = pd.DataFrame(tes_dict)

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))

# with open(path_data_train, "wb") as file_object:
#     pickle.dump(tra_dict, file_object)
    
# with open(path_data_test, "wb") as file_object:
#     pickle.dump(tes_dict, file_object)

# tra_df.to_csv(path_data_train, sep=",", header=False, index=False, encoding='utf-8')
# tes_df.to_csv(path_data_test, sep=",", header=False, index=False, encoding='utf-8')   

print("dataset: ", datasets_name)
print("model: AutoSecRec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

item_num:  6208
train_seq_num & user_num:  42868
test_seq_num:  7272
#interactions:  50140
sequence average length:  1.1696370252869273
dataset:  Cell_Phones_and_Accessories
model: AutoSecRec
2023-12-23 14:28:45
done


In [4]:
len(tes[2])

48335

## MAERec SIGIR 2023 Graph Masked Autoencoder for Sequential Recommendation

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np
import copy
from scipy.sparse import csr_matrix, coo_matrix

meta_dataset = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + meta_dataset +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'


if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data_all = data_all[['sessionID', 'itemID']]


# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}


item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all: 
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]

tra_seqs = []
tes_seqs = []
        
# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[x1, x2, x3], [label...]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra_seqs.append(new_seq)
    
item_count =  item_num-1
user_count = len(tra_seqs)


# test_set [[x_id ...],[label...]
tes = [[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes_seqs.append(new_seq)
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)       


train_data_path = 'MAERec/' + meta_dataset + '/'
if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)

user = list()
r, c, d = list(), list(), list()
# k=4
for i, seq in enumerate(tra_seqs):
    for dist in range(1, 3):
        if dist >= len(seq): break;
        r += copy.deepcopy(seq[+dist:])
        c += copy.deepcopy(seq[:-dist])
        r += copy.deepcopy(seq[:-dist])
        c += copy.deepcopy(seq[+dist:])
d = np.ones_like(r)
iigraph = csr_matrix((d, (r, c)), shape=(item_num, item_num))
print('Constructed i-i graph, density=%.6f' % (len(d) / ((item_num) ** 2)))
neg_seqs = []
for t_seq in tes_seqs:
    non_lab = t_seq[-1]
    ran_list = np.random.randint(1, item_num, 99)
    ran_list = [1 if i == non_lab else i for i in ran_list]
    neg_seqs.append(ran_list)

with open(train_data_path + 'trn', 'wb') as fs:
    pickle.dump(iigraph, fs)

with open(train_data_path + 'seq', 'wb') as fs:
    pickle.dump(tra_seqs, fs)
    
with open(train_data_path + 'tst', 'wb') as fs:
    pickle.dump(tes_seqs, fs)
    
with open(train_data_path + 'neg', 'wb') as fs:
    pickle.dump(neg_seqs, fs)
    
print("dataset: ", meta_dataset)
print("Model: MAERec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

item_num:  6208
train_seq_num & user_num:  7598
test_seq_num:  7272
#interactions:  50140
sequence average length:  6.599105027638853
Constructed i-i graph, density=0.003265
dataset:  Cell_Phones_and_Accessories
Model: MAERec
2023-12-21 20:48:26
done


## senti backup 有每个attribute下，user-item交互的矩阵，用于attribute-graph update

In [ ]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Cell_Phones_and_Accessories'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]

attr_num = len(attr_list)

# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}
attr1_all = {}
attr2_all = {}
attr3_all = {}
attr4_all = {}
attr5_all = {}
attr6_all = {}
attr7_all = {}

item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    attr1_opin = row[attr_list[0]]
    attr2_opin = row[attr_list[1]]
    attr3_opin = row[attr_list[2]]
    attr4_opin = row[attr_list[3]]
    attr5_opin = row[attr_list[4]]
    attr6_opin = row[attr_list[5]]
    attr7_opin = row[attr_list[6]]
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id].append(attr7_opin)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id] = []
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id] = []
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id] = []
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id] = []
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id] = []
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id] = []
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id] = []
        attr7_all[sess_id].append(attr7_opin)


# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u_id ...], [x_t-1...], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}

# dict {item_id:opin_id}
attr1_item_opin = {}
attr2_item_opin = {}
attr3_item_opin = {}
attr4_item_opin = {}
attr5_item_opin = {}
attr6_item_opin = {}
attr7_item_opin = {}



for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    a1_opin = attr1_all[sess_temp][:-1]
    a2_opin = attr2_all[sess_temp][:-1]
    a3_opin = attr3_all[sess_temp][:-1]
    a4_opin = attr4_all[sess_temp][:-1]
    a5_opin = attr5_all[sess_temp][:-1]
    a6_opin = attr6_all[sess_temp][:-1]
    a7_opin = attr7_all[sess_temp][:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x, a1, a2, a3, a4, a5, a6, a7 in zip(train_seqs_t, a1_opin, a2_opin,a3_opin,a4_opin,a5_opin,a6_opin,a7_opin):
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
                attr1_item_opin[item_dict[x]].append(a1)
                attr2_item_opin[item_dict[x]].append(a2)
                attr3_item_opin[item_dict[x]].append(a3)
                attr4_item_opin[item_dict[x]].append(a4)
                attr5_item_opin[item_dict[x]].append(a5)
                attr6_item_opin[item_dict[x]].append(a6)
                attr7_item_opin[item_dict[x]].append(a7)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            attr1_item_opin[item_num] = []
            attr2_item_opin[item_num] = []
            attr3_item_opin[item_num] = []
            attr4_item_opin[item_num] = []
            attr5_item_opin[item_num] = []
            attr6_item_opin[item_num] = []
            attr7_item_opin[item_num] = []
            attr1_item_opin[item_num].append(a1)
            attr2_item_opin[item_num].append(a2)
            attr3_item_opin[item_num].append(a3)
            attr4_item_opin[item_num].append(a4)
            attr5_item_opin[item_num].append(a5)
            attr6_item_opin[item_num].append(a6)
            attr7_item_opin[item_num].append(a7)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
#     预测时考虑last item
    tra[1].append(new_seq[:-1])
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

fre_item_mat = np.zeros((item_count, item_count), dtype=int)
# 全部的共现关系Original 
for sess in train_sess.keys():
    item_seq = np.unique(train_sess[sess])
    for it1 in item_seq:
        for it2 in item_seq:
            if it1 != it2:
#                 print(sess)
                fre_item_mat[int(it1)-1, int(it2)-1] +=1 
                
fre_user_mat = np.zeros((user_count, user_count), dtype=int)
# 全部的共现关系Original 
for item_t in train_items.keys():
    user_seq = np.unique(train_items[item_t])
    for it1 in user_seq:
        for it2 in user_seq:
            if it1 != it2:
                fre_user_mat[int(it1)-1, int(it2)-1] +=1 

# building user_fre & item_fre csr_matrix
def spar_graph(fre_mat):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for i_list in fre_mat:
        i_temp_list = np.nonzero(i_list)[0].tolist()
#         if len(i_temp_list) == 0:
#             non_co += 1
        indices += i_temp_list
        all_data += [1]*len(i_temp_list)
        indptr.append(indptr[-1]+len(i_temp_list))
#     print('item without co-occurren: ',str(non_co))
    # indptr:session长度累加和; indices:item_id 减1, 由每个session内item组成; data:item在session内的权重，全部为1.
    coo_mat = (all_data, indices, indptr)
    return coo_mat

item_co_mat = spar_graph(fre_item_mat)
user_co_mat = spar_graph(fre_user_mat)


# test_set [[u_id ...], [x_t-1...], [label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        tes_seq_num += 1
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes[0].append(sess_temp)
    #     预测时考虑last item
        tes[1].append(new_seq[:-1])
        tes[2].append(item_dict[all_seqs[-1]])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num

#  obtain user-item interaction sparsity matrix in each attribute
def spar_matrix(attr, tra_seq):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for sess_temp in attr.keys():
        op_list = attr[sess_temp][:-1]
        it_list = tra_seq[sess_temp]
        np_op = np.array(op_list)
        np_it = np.array(it_list)
        op_f = np_op[np.nonzero(np_op)[0]]
        it_f = np_it[np.nonzero(np_op)[0]]

        item_l = np.unique(it_f)
        op_l = op_f[np.unique(it_f, return_index=True)[1]]
        indices += (item_l-1).tolist()
        all_data += op_l.tolist()
        indptr.append(indptr[-1] + len(item_l.tolist()))
    results = (all_data, indices, indptr)
    return results

attr1_mat = spar_matrix(attr1_all, train_sess)
attr2_mat = spar_matrix(attr2_all, train_sess)
attr3_mat = spar_matrix(attr3_all, train_sess)
attr4_mat = spar_matrix(attr4_all, train_sess)
attr5_mat = spar_matrix(attr5_all, train_sess)
attr6_mat = spar_matrix(attr6_all, train_sess)
attr7_mat = spar_matrix(attr7_all, train_sess)






# 返回 [[u1,u2],[attr1_opin, attr1_opin]]
def formulate_seq(dict_ui, dict_op):
    results = [[],[]]
    temp_res = [[],[]]
    max_len =0
    for x in dict_ui.keys():
        seqs_id = dict_ui[x]
        opin_id = dict_op[x]
        new_us_seq = []
        new_op_seq = []
        for id_t, op_t in zip(seqs_id, opin_id):
            if op_t != 0 and id_t not in new_us_seq:
                new_us_seq.append(id_t)
                new_op_seq.append(op_t)
        if len(new_us_seq)==0:
            new_us_seq.append(0)
            new_op_seq.append(0)
        if len(new_us_seq)>max_len:
            max_len = len(new_us_seq)
        temp_res[0].append(new_us_seq)
        temp_res[1].append(new_op_seq)
    for ui_seq, op_seq in zip(temp_res[0],temp_res[1]):
        pad_len = max_len - len(ui_seq)
        pad_ui_seq = ui_seq + [0]*pad_len
        pad_op_seq = op_seq + [0]*pad_len
        results[0].append(pad_ui_seq)
        results[1].append(pad_op_seq)
#     print('max_len:',str(max_len))
    return results

# u_seq = {1:[1,2,3,1],2:[4,5], 3:[7,8,9]}
# u_op = {1:[45,56,0,45],2:[66,0], 3:[0,33,0]}
# # [[[1,2],[4],[8]],[[45,56],[66],[33]]]
# formulate_seq(u_seq,u_op)

ui_1 = formulate_seq(train_sess,attr1_all)
ui_2 = formulate_seq(train_sess,attr2_all)
ui_3 = formulate_seq(train_sess,attr3_all)
ui_4 = formulate_seq(train_sess,attr4_all)
ui_5 = formulate_seq(train_sess,attr5_all)
ui_6 = formulate_seq(train_sess,attr6_all)
ui_7 = formulate_seq(train_sess,attr7_all)
iu_1 = formulate_seq(train_items,attr1_item_opin)
iu_2 = formulate_seq(train_items,attr2_item_opin)
iu_3 = formulate_seq(train_items,attr3_item_opin)
iu_4 = formulate_seq(train_items,attr4_item_opin)
iu_5 = formulate_seq(train_items,attr5_item_opin)
iu_6 = formulate_seq(train_items,attr6_item_opin)
iu_7 = formulate_seq(train_items,attr7_item_opin)

print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './datasets/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
tra_save = (tra[0], tra[1], tra[2],item_co_mat,user_co_mat, attr1_mat, attr2_mat, attr3_mat, attr4_mat, attr5_mat, attr6_mat, attr7_mat, ui_1, ui_2, ui_3, ui_4, ui_5, ui_6, ui_7, iu_1, iu_2, iu_3, iu_4, iu_5, iu_6, iu_7)
tes_save = (tes[0], tes[1], tes[2])

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

In [4]:
[1,2,3,4]+2*[0]

[1, 2, 3, 4, 0, 0]

## senti backup 都是用矩阵来进行运算，-> heterogeneity

In [ ]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Beauty'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]

attr_num = len(attr_list)

# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}
attr1_all = {}
attr2_all = {}
attr3_all = {}
attr4_all = {}
attr5_all = {}
attr6_all = {}
attr7_all = {}

item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    attr1_opin = row[attr_list[0]]
    attr2_opin = row[attr_list[1]]
    attr3_opin = row[attr_list[2]]
    attr4_opin = row[attr_list[3]]
    attr5_opin = row[attr_list[4]]
    attr6_opin = row[attr_list[5]]
    attr7_opin = row[attr_list[6]]
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id].append(attr7_opin)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id] = []
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id] = []
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id] = []
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id] = []
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id] = []
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id] = []
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id] = []
        attr7_all[sess_id].append(attr7_opin)


# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u_id ...], [x_t-1...], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
#     预测时考虑last item
    tra[1].append(new_seq[-2])
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

fre_item_mat = np.zeros((item_count, item_count), dtype=int)
# 全部的共现关系Original 
for sess in train_sess.keys():
    item_seq = np.unique(train_sess[sess])
    for it1 in item_seq:
        for it2 in item_seq:
            if it1 != it2:
#                 print(sess)
                fre_item_mat[int(it1)-1, int(it2)-1] +=1 
                
fre_user_mat = np.zeros((user_count, user_count), dtype=int)
# 全部的共现关系Original 
for item_t in train_items.keys():
    user_seq = np.unique(train_items[item_t])
    for it1 in user_seq:
        for it2 in user_seq:
            if it1 != it2:
                fre_user_mat[int(it1)-1, int(it2)-1] +=1 

# building user_fre & item_fre csr_matrix
def spar_graph(fre_mat):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for i_list in fre_mat:
        i_temp_list = np.nonzero(i_list)[0].tolist()
#         if len(i_temp_list) == 0:
#             non_co += 1
        indices += i_temp_list
        all_data += [1]*len(i_temp_list)
        indptr.append(indptr[-1]+len(i_temp_list))
#     print('item without co-occurren: ',str(non_co))
    # indptr:session长度累加和; indices:item_id 减1, 由每个session内item组成; data:item在session内的权重，全部为1.
    coo_mat = (all_data, indices, indptr)
    return coo_mat

item_co_mat = spar_graph(fre_item_mat)
user_co_mat = spar_graph(fre_user_mat)


# test_set [[u_id ...], [x_t-1...], [label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        tes_seq_num += 1
        tes[0].append(sess_temp)
    #     预测时考虑last item
        tes[1].append(item_dict[all_seqs[-2]])
        tes[2].append(item_dict[all_seqs[-1]])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num

#  obtain user-item interaction sparsity matrix in each attribute
def spar_matrix(attr, tra_seq):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for sess_temp in attr.keys():
        op_list = attr[sess_temp][:-1]
        it_list = tra_seq[sess_temp]
        np_op = np.array(op_list)
        np_it = np.array(it_list)
        op_f = np_op[np.nonzero(np_op)[0]]
        it_f = np_it[np.nonzero(np_op)[0]]

        item_l = np.unique(it_f)
        op_l = op_f[np.unique(it_f, return_index=True)[1]]
        indices += (item_l-1).tolist()
        all_data += op_l.tolist()
        indptr.append(indptr[-1] + len(item_l.tolist()))
    results = (all_data, indices, indptr)
    return results

attr1_mat = spar_matrix(attr1_all, train_sess)
attr2_mat = spar_matrix(attr2_all, train_sess)
attr3_mat = spar_matrix(attr3_all, train_sess)
attr4_mat = spar_matrix(attr4_all, train_sess)
attr5_mat = spar_matrix(attr5_all, train_sess)
attr6_mat = spar_matrix(attr6_all, train_sess)
attr7_mat = spar_matrix(attr7_all, train_sess)

print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './datasets/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
tra_save = (tra[0], tra[1], tra[2],item_co_mat,user_co_mat, attr1_mat, attr2_mat, attr3_mat, attr4_mat, attr5_mat, attr6_mat, attr7_mat)
tes_save = (tes[0], tes[1], tes[2])

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")


## NARM

In [2]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Cell_Phones_and_Accessories'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data_all = data_all[['sessionID', 'itemID']]


# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}


item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        if len(sess_all[sess_id]) < 21: 
            sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)


# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[x1, x2, x3], [label...]
tra = [[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(new_seq[:-1])
#     预测时考虑last item
    tra[1].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(new_seq[:-1])
    #     预测时考虑last item
        tes[1].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './NARM/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
tra_save = (tra[0], tra[1])
tes_save = (tes[0], tes[1])

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("NARM dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")


item_num:  10346
train_seq_num & user_num:  27777
test_seq_num:  27566
#interactions:  189133
sequence average length:  6.8089786514022395
NARM dataset:  Cell_Phones_and_Accessories
2023-12-20 18:09:01
done


## BERT4Rec

In [5]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Cell_Phones_and_Accessories'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty' or meta_dataset == 'Beauty_4':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    # Price, Style, Size, Fabric, Brand, Quality, Color
    attr_list = ['Price', 'Style', 'Size', 'Fabric', 'Brand', 'Quality', 'Color']

data_all = pd.read_csv(data_path)
data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]
data_all = data_all[['sessionID', 'itemID']]


# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}


item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)


# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[user1,user2],[x1, x2, x3], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[user1,user2],[x_id ...],[label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './BERT4Rec/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
tra_save = (tra[0], tra[1], tra[2])
tes_save = (tes[0], tes[1], tes[2])

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("BERT4Rec dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")


item_num:  10388
train_seq_num & user_num:  27777
test_seq_num:  27645
#interactions:  193599
sequence average length:  6.969759153256291
NARM dataset:  Cell_Phones_and_Accessories
2023-12-20 17:44:22
done
